# Лабораторная работа №2

## Реализация глубокой нейронной сети

В работе предлагается использовать набор данных _notMNIST_, который состоит из изображений размерностью 28×28 первых 10 букв латинского алфавита (_A_ ... _J_, соответственно). Обучающая выборка содержит порядка 500 тыс. изображений, а тестовая – около 19 тыс.

Данные можно скачать по ссылке:

* https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz (большой набор данных);

* https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz (маленький набор данных);

Описание данных на английском языке доступно по ссылке:
http://yaroslavvb.blogspot.sg/2011/09/notmnist-dataset.html

### Задание 1

Реализуйте полносвязную нейронную сеть с помощью библиотеки _TensorFlow_. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (_Stochastic Gradient Descent_, _SGD_). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

os.chdir(BASE_DIR)

In [0]:
import pandas as pd

dataframe = pd.read_pickle("./large.pkl")

In [4]:
dataframe['data'].shape

(461946,)

In [5]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.2.0rc2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: protobuf, termcolor, wrapt, h5py, wheel, astunparse, scipy, six, gast, absl-py, tensorboard, opt-einsum, google-pasta, grpcio, numpy, tensorflow-estimator, keras-preprocessing
Required-by: 


In [0]:
import tensorflow as tf

In [0]:
import numpy as np

In [0]:
# train_df = dataframe.sample(frac = 0.7)
# test_df = dataframe.drop(train_df.index)

# train_df = train_df.reset_index()
# test_df = test_df.reset_index()

In [0]:
# train_dataset = tf.data.Dataset.from_tensor_slices((np.asarray(list(train_df['data'])), train_df['label']))
# test_dataset = tf.data.Dataset.from_tensor_slices((np.asarray(list(test_df['data'])), test_df['label']))

In [0]:
# dataset = tf.data.Dataset.from_tensors((np.asarray(list(dataframe['data'])), dataframe['label']))

In [11]:
x = np.asarray(list(dataframe['data']))[..., np.newaxis]

x = tf.keras.utils.normalize(x, axis = 1)

x.shape

(461946, 28, 28, 1)

In [12]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(dataframe['label'].astype('category').cat.codes.astype('int32'))

y.shape

(461946, 10)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = tf.keras.Sequential()

model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10))

In [0]:
def custom_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(
    y_true, y_pred, from_logits=True)

model.compile(optimizer='sgd',
              loss=custom_loss,
              metrics=['categorical_accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [0]:
batch_size = 128

In [0]:
r = 3608

In [18]:
model.fit(x = x[:r * batch_size], y = y[:r * batch_size], epochs = 50, batch_size = batch_size,
          validation_split = 0.15)

Epoch 1/50
3067/3067 [==============================] - 13s 4ms/step - loss: 0.9794 - categorical_accuracy: 0.7194 - val_loss: 7.9179 - val_categorical_accuracy: 0.2404
Epoch 2/50
3067/3067 [==============================] - 12s 4ms/step - loss: 0.4839 - categorical_accuracy: 0.8569 - val_loss: 6.6196 - val_categorical_accuracy: 0.2464
Epoch 3/50
3067/3067 [==============================] - 12s 4ms/step - loss: 0.4193 - categorical_accuracy: 0.8739 - val_loss: 6.7106 - val_categorical_accuracy: 0.2584
Epoch 4/50
3067/3067 [==============================] - 12s 4ms/step - loss: 0.3838 - categorical_accuracy: 0.8840 - val_loss: 6.7257 - val_categorical_accuracy: 0.2600
Epoch 5/50
3067/3067 [==============================] - 12s 4ms/step - loss: 0.3604 - categorical_accuracy: 0.8905 - val_loss: 7.3149 - val_categorical_accuracy: 0.2747
Epoch 6/50
3067/3067 [==============================] - 13s 4ms/step - loss: 0.3431 - categorical_accuracy: 0.8954 - val_loss: 7.3590 - val_categorical_acc

### Задание 2

Как улучшилась точность классификатора по сравнению с логистической регрессией?

### Задание 3

Используйте регуляризацию и метод сброса нейронов (_dropout_) для борьбы с переобучением. Как улучшилось качество классификации?

### Задание 4

Воспользуйтесь динамически изменяемой скоростью обучения (_learning rate_). Наилучшая точность, достигнутая с помощью данной модели составляет 97.1%. Какую точность демонстрирует Ваша реализованная модель?